In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\Maza\\Desktop\\job_aplication_agent'

In [8]:
from dataclasses import dataclass, field
from pathlib import Path
from typing import Union

@dataclass(frozen=True)
class CVConfig:
    CV_file: Union[Path, str]
    edited_CV: Union[Path, str]
    job_description: Union[Path, str]
    qualifications: Union[Path, str]
    contact_info: Union[Path, str]
    about_me: Union[Path, str]
    education: Union[Path, str]
    experience: Union[Path, str]
    projects: Union[Path, str]
    certifications: Union[Path, str]
    model: str
    CV_prompt_templates: dict

    def __post_init__(self):
        # Convert string paths to Path objects
        object.__setattr__(self, 'CV_file', Path(self.CV_file))
        object.__setattr__(self, 'edited_CV', Path(self.edited_CV))
        object.__setattr__(self, 'job_description', Path(self.job_description))
        object.__setattr__(self, 'qualifications', Path(self.qualifications))
        object.__setattr__(self, 'contact_info', Path(self.contact_info))
        object.__setattr__(self, 'about_me', Path(self.about_me))
        object.__setattr__(self, 'education', Path(self.education))
        object.__setattr__(self, 'experience', Path(self.experience))
        object.__setattr__(self, 'projects', Path(self.projects))
        object.__setattr__(self, 'certifications', Path(self.certifications))



In [9]:
from dotenv import load_dotenv
from pathlib import Path


from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser,JsonOutputParser

from job_aplication_agent.utils.common import  load_json,read_yaml,read_file,save_file
from job_aplication_agent import logger
from job_aplication_agent.constants import *
load_dotenv()

True

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        models_filepath = MODELS_FILE_PATH,
        prompt_template = PROMPT_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.models = read_yaml(models_filepath)
        self.prompt_template = read_yaml(prompt_template)
    
        

    
    def get_CV_config(self) -> CVConfig:
        """
        
        Returns:
            
        """
        
        CV_config = self.config.CV
        prompt_templates = self.prompt_template
        
        create_CV_config = CVConfig(
            CV_file= CV_config.CV_file,
            edited_CV = CV_config.edited_CV,
            job_description= CV_config.job_description,
            qualifications= CV_config.qualifications,
            contact_info=CV_config.contact_info,
            about_me=CV_config.about_me,
            education=CV_config.education,
            experience=CV_config.experience,
            projects=CV_config.projects,
            certifications=CV_config.certifications,
            model= self.models.Llama3,
            CV_prompt_templates = prompt_templates.cv_prompt_templates

           
        ) 

        return create_CV_config

In [21]:
class GetCV:
    def __init__(self,config:CVConfig):
        self.config = config
        self.CV_prompt_templates = config.CV_prompt_templates
        try:
            self.CV = read_file(config.CV_file)
            logger.info(f"CV loaded")
        except Exception as e:
            logger.error(f"Loading CV error {e}")

        try:
            self.qualifications = read_file(config.qualifications)
            logger.info(f"Qualifications loaded")
        except Exception as e:
            logger.error(f"Loading qualifications error {e}")

        try:
            self.job_description = read_file(config.job_description)
            logger.info(f"Job description loaded")
        except Exception as e:
            logger.error(f"Loading job description error {e}")

    def create_CV_agents(self):
        llm = ChatGroq(temperature=0,model_name= self.config.model)
        logger.info(f"Llm initialized")

        #Agents
        try:
            contact_info_prompt = PromptTemplate(
                template= self.CV_prompt_templates.contact_info_template,
                input_variables=["CONTACT_INFO","CV"]
            )
            self.contact_info_agent = contact_info_prompt | llm | StrOutputParser()
            logger.info("Contact info Agent created")
        except Exception as e:
            logger.error("Creating contact info agent error: {e}")

        try:
            summary_prompt = PromptTemplate(
                template= self.CV_prompt_templates.summary_template,
                input_variables=["JOB_DESCRIPTION","QUALIFICATIONS","CV"]
            )
            self.summary_agent = summary_prompt | llm | StrOutputParser()
            logger.info("Summary Agent created")
        except Exception as e:
            logger.error("Creating summary agent error: {e}")

        try:
            skills_prompt = PromptTemplate(
                template= self.CV_prompt_templates.skills_template,
                input_variables=["JOB_DESCRIPTION","QUALIFICATIONS","CV"]
            )
            self.skills_agent = skills_prompt | llm | StrOutputParser()
            logger.info("Skills Agent created")
        except Exception as e:
            logger.error("Creating skills agent error: {e}")

        try:
            experience_prompt = PromptTemplate(
                template= self.CV_prompt_templates.experience_template,
                input_variables=["JOB_DESCRIPTION","EXPERIENCE","CV"]
            )
            self.experience_agent = experience_prompt | llm | StrOutputParser()
            logger.info("Experience Agent created")
        except Exception as e:
            logger.error("Creating experience agent error: {e}")
        
        try:
            projects_prompt = PromptTemplate(
                template= self.CV_prompt_templates.projects_template,
                input_variables=["PROJECTS","CV"]
            )
            self.projects_agent = projects_prompt | llm | StrOutputParser()
            logger.info("Projects Agent created")
        except Exception as e:
            logger.error("Creating projects agent error: {e}")
        
        try:
            education_prompt = PromptTemplate(
                template= self.CV_prompt_templates.education_template,
                input_variables=["EDUCATION","CV"]
            )
            self.education_agent = education_prompt | llm | StrOutputParser()
            logger.info("Education Agent created")
        except Exception as e:
            logger.error("Creating education agent error: {e}")
    
        try:
            certification_prompt = PromptTemplate(
                template= self.CV_prompt_templates.certifications_template,
                input_variables=["CERTIFICATIONS","CV"]
            )
            self.certifications_agent = certification_prompt | llm | StrOutputParser()
            logger.info("Certification Agent created")
        except Exception as e:
            logger.error("Creating certification agent error: {e}")
        
        try:
            about_me_prompt = PromptTemplate(
                template= self.CV_prompt_templates.about_me_template,
                input_variables=["ABOUT_ME","CV"]
            )
            self.about_me_agent = about_me_prompt | llm | StrOutputParser()
            logger.info("About me Agent created")
        except Exception as e:
            logger.error("Creating about me agent error: {e}")
    
  
    def get_contact_info(self):
        logger.info(f"Updating contact info")
        try:
            contact_info = read_yaml(self.config.contact_info)
            logger.info(f"Contact info loaded")
        except Exception as e:
            logger.error(f"Loading contact info error {e}")
        
        try:
            cv_conatct_info = self.contact_info_agent.invoke({
            "CONTACT_INFO":contact_info,
            "CV":self.CV
            })
            self.CV = cv_conatct_info
            save_file(self.config.edited_CV,cv_conatct_info)
            logger.info(f"Contact info updated")
        except Exception as e:
            logger.error(f"Updating contact info error {e}")
    
    def get_summary(self):
        logger.info(f"Updating summary")
        try:
            cv_summary = self.summary_agent.invoke({
                "JOB_DESCRIPTION":self.job_description,
                "QUALIFICATIONS":self.qualifications,
                "CV":self.CV
                })
            self.CV = cv_summary
            save_file(self.config.edited_CV,cv_summary)
            logger.info(f"Summary updated")
        except Exception as e:
            logger.error(f"Updating summary error {e}")
    
    def get_skills(self):
        logger.info(f"Updating skills")
        try:
            cv_skills = self.skills_agent.invoke({
                "JOB_DESCRIPTION":self.job_description,
                "QUALIFICATIONS":self.qualifications,
                "CV":self.CV
                })
            self.CV = cv_skills
            save_file(self.config.edited_CV,cv_skills)
            logger.info(f"Skills updated")
        except Exception as e:
            logger.error(f"Updating skills error {e}")

    def get_experience(self):
        logger.info(f"Updating experinece")
        try:
            experience = read_file(self.config.experience)
            logger.info(f"Experience loaded")
        except Exception as e:
            logger.error(f"Loading experience error {e}")
            
        try:
            cv_experinece = self.experience_agent.invoke({
            "JOB_DESCRIPTION":self.job_description,
            "EXPERIENCE":experience,
            "CV":self.CV
            })
            self.CV = cv_experinece
            save_file(self.config.edited_CV,cv_experinece)
            logger.info(f"Experinece updated")
        except Exception as e:
            logger.error(f"Updating experience error {e}")

    def get_projects(self):
        logger.info(f"Updating projects")
        try:
            projects = read_file(self.config.projects)
            logger.info(f"Projects loaded")
        except Exception as e:
            logger.error(f"Loading projects error {e}")
            
        try:
            cv_projects= self.projects_agent.invoke({
            "PROJECTS":projects,
            "CV":self.CV
            })
            self.CV = cv_projects
            save_file(self.config.edited_CV,cv_projects)
            logger.info(f"Projects updated")
        except Exception as e:
            logger.error(f"Updating projects error {e}")


    def get_certifications(self):
        logger.info(f"Updating cerification")
        try:
            certifications = read_file(self.config.certifications)
            logger.info(f"Certifications loaded")
        except Exception as e:
            logger.error(f"Loading certifications error {e}")
        
        try:
            cv_certifications = self.certifications_agent.invoke({
                "CERTIFICATIONS":certifications,
                "CV": self.CV
            })
            self.CV = cv_certifications
            save_file(self.config.edited_CV,cv_certifications)
            logger.info(f"Certifications updated")
        except Exception as e:
            logger.error(f"Updating certifications error {e}")
            

    def get_education(self):
        logger.info(f"Updating education")
        try:
            education = read_file(self.config.education)
            logger.info(f"Education loaded")
        except Exception as e:
            logger.error(f"Loading education error {e}")
            
        try:
            cv_education = self.education_agent.invoke({
                "EDUCATION":education,
                "CV": self.CV
            })
            self.CV = cv_education
            save_file(self.config.edited_CV,cv_education)
            logger.info(f"Education updated")
        except Exception as e:
            logger.error(f"Updating education error {e}")
                

    def get_about_me(self):
        logger.info(f"Updating about_me")
        try:
            about_me = read_file(self.config.about_me)
            logger.info(f"About me loaded")
        except Exception as e:
            logger.error(f"Loading about me error {e}")
            
        try:
            cv_about_me = self.about_me_agent.invoke({
                "ABOUT_ME":about_me,
                "CV":self.CV
                })
            self.CV = cv_about_me
            save_file(self.config.edited_CV,cv_about_me)
            logger.info(f"About me updated")
        except Exception as e:
            logger.error(f"Updating about me error {e}")
    
    

In [22]:
config = ConfigurationManager()
cv_config = config.get_CV_config()
get_cv = GetCV(cv_config)
get_cv.create_CV_agents()
get_cv.get_contact_info()
get_cv.get_summary()
get_cv.get_skills()
get_cv.get_experience()
get_cv.get_projects()
get_cv.get_education()
get_cv.get_certifications()
get_cv.get_about_me()

[2024-07-14 13:40:05,972: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2024-07-14 13:40:05,976: INFO: common: yaml file: models.yaml loaded successfully:]
[2024-07-14 13:40:05,986: INFO: common: yaml file: prompt_template.yaml loaded successfully:]
[2024-07-14 13:40:05,988: INFO: 2608794138: CV loaded:]
[2024-07-14 13:40:05,990: INFO: 2608794138: Qualifications loaded:]
[2024-07-14 13:40:05,992: INFO: 2608794138: Job description loaded:]
[2024-07-14 13:40:06,465: INFO: 2608794138: Llm initialized:]
[2024-07-14 13:40:06,465: INFO: 2608794138: Contact info Agent created:]
[2024-07-14 13:40:06,465: INFO: 2608794138: Summary Agent created:]
[2024-07-14 13:40:06,465: INFO: 2608794138: Skills Agent created:]
[2024-07-14 13:40:06,465: INFO: 2608794138: Experience Agent created:]
[2024-07-14 13:40:06,465: INFO: 2608794138: Projects Agent created:]
[2024-07-14 13:40:06,481: INFO: 2608794138: Education Agent created:]
[2024-07-14 13:40:06,481: INFO: 2608794138: Certificatio

In [17]:
get_cv.get_education()

[2024-07-13 10:19:05,660: INFO: 754182642: Updating education:]
[2024-07-13 10:19:05,672: INFO: 754182642: Education loaded:]
[2024-07-13 10:19:12,056: INFO: _client: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK":]
[2024-07-13 10:19:12,059: INFO: 754182642: Education updated:]


[2024-07-10 10:59:25,203: INFO: 346236587: Updating experinece:]
prompt_text_inputs\experience.txt
prompt_text_inputs\experience.txt
[2024-07-10 10:59:25,207: ERROR: 346236587: Loading experience error mapping values are not allowed here
  in "prompt_text_inputs\experience.txt", line 3, column 21:]
[2024-07-10 10:59:25,208: ERROR: 346236587: Updating experience error cannot access local variable 'experinece' where it is not associated with a value:]


In [24]:
def read_file(file:Path) -> str:
    with open(file, 'r', encoding='utf-8') as file:
        # Read the entire content of the file
        content = file.read()
    
    return content

In [16]:
test = read_file(cv_config.experience)

In [14]:
test

"February 2024 to April 2024:\n    Internship in Mesh Firm as Machine learning Engineer\n    Responsabilities:\n        Developed vectro database fro Retrival Augmented Geneartion application using Pinecone\n        Created API with Fast API with post and get methods for LLM chat application, using Langchain, Langraph, OpenAI and Lama3\n        Monitored Llm agents functionality and token usage with Langsimth\n\nAugust 2022 to September 2023\n    Part time job in Precure as Data Engineer\n    Responsabilities:\n        Established a CI/CD pipeline using Jenkins for streamlined project development and deployment.\n        Developed a CNN system leveraging TensorFlow to detect and exclude noisy EMG signals        \n        Configured and managed Linux servers for efficient data processing and analysis\n        Conducted a comparative analysis of EMG signals from different materials \n        using Python and LaTeX, to improve material durability.\n        Designed and implemented Docker 

In [11]:
with open('prompt_text_inputs\experience.txt', 'r') as file:
    # Read the content of the file
    file_content = file.read()

In [10]:
file_content

"February 2024 to April 2024:\n    Internship in Mesh Firm as Machine learning Engineer\n    Responsabilities:\n        Developed vectro database fro Retrival Augmented Geneartion application using Pinecone\n        Created API with Fast API with post and get methods for LLM chat application, using Langchain, Langraph, OpenAI and Lama3\n        Monitored Llm agents functionality and token usage with Langsimth\n\nAugust 2022 to September 2023\n    Part time job in Precure as Data Engineer\n    Responsabilities:\n        Established a CI/CD pipeline using Jenkins for streamlined project development and deployment.\n        Developed a CNN system leveraging TensorFlow to detect and exclude noisy EMG signals        \n        Configured and managed Linux servers for efficient data processing and analysis\n        Conducted a comparative analysis of EMG signals from different materials \n        using Python and LaTeX, to improve material durability.\n        Designed and implemented Docker 